# BigMPI4py tutorial

BigMPI4py is a module developed based on Lisando Dalcin's implementation of Message Passing 
Interface (MPI for short) for python, MPI4py (https://mpi4py.readthedocs.io), which allows for
parallelization of data structures within python code. 

Although many of the common cases of parallelization can be solved with MPI4py alone, there
are cases were big data structures cannot be distributed across cores within MPI4py 
infrastructure. This limitation is well known for MPI4py and happens due to the fact that MPI 
calls have a buffer limitation of 2GB entries. 

In order to solve this problem, some solutions exist, like dividing the datasets in *chunks* that
satisfy the data size criterion, or using other MPI implementations such as BigMPI 
(https://github.com/jeffhammond/BigMPI). BigMPI requires both understanding
the syntax of BigMPI, as well as having to adapt python scripts to BigMPI, which can be 
difficult and requires knowledge of C-based programming languages, of which many users have a 
lack of. Then, the *chunking* strategy can be used in Python, but has to be adapted manually for 
data types and, in many cases, the number of elements that each node will receive which, in order
to circumvent the 2 GB problem, can be difficult.

BigMPI4py adapts the *chunking* strategy of data, being able to automatically distribute 
the most common python data types used in python, such as numpy arrays, pandas dataframes, lists, nested lists, 
or lists of arrays and dataframes. Therefore, users of BigMPI4py can automatically parallelize their 
pipelines by calling BigMPI4py's functions with their data.

Look up our paper in bioRxiv to see how the software works.
https://www.biorxiv.org/content/early/2019/01/17/517441

If you find this software useful, please cite us:

Alex M. Ascensión and Marcos J. Araúzo-Bravo. BigMPI4py: Python module for parallelization of Big Data objects; bioRxiv, (2019). doi: 10.1101/517441. 

## Running a script with BigMPI4py

Like MPI4py, any script has to be run within bash by calling `mpirun`. Let's suppose the script is saved into `myscript.py`. Then, the work can be run with the sollowing command in the console (or in any console, like the one from Pycharm):

`mpirun -np XX python myscript.py`

`np` is the number of cores to be used. If `python` is not added to the PATH, you can add the full path, or export to the PATH variable. If you are running `python` via conda, the path is `PATH-TO-CONDA/bin/python`. You can check conda location writing `which conda`.


## Running a code using Jupyter notebooks
Thanks to Ipython magic you won't need to run this tutorial using the console! Still, I will add a little example of how to run a piece of code through this notebook. 

Despite not using the console directly, you will still have to call MPI, which needs the python script to run. To run the script using Ipython, we will output the content of a cell to a file and run this file through the notebook.

The first step is to assign a directory where all the files of this tutorial will be saved and run.

<span style="color:red;" ><b>RUN THESE TWO CELLS TO SET SOME GENERAL VARIABLES!</b></span>


In [ ]:
import os,sys,inspect
import numpy as np
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))
        
cwd = !pwd

tutorial_directory =  cwd[0] + '/BigMPI4py_tutorial/' # <-- Change the location of the tutorial files here!
number_cores = 8  # <-- Change the number of cores here!

if not os.path.exists(tutorial_directory): os.mkdir(tutorial_directory)
    


In [ ]:
file_header = '''
import sys
import os
import time

from mpi4py import MPI
import numpy as np
import pandas as pd

import bigmpi4py as BM

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 10)

comm = MPI.COMM_WORLD
size, rank = comm.Get_size(), comm.Get_rank()
'''

Now that the directory is set, we will create a variable for the name of the .py file the contets of a cell will be saved on; we will create a sample of code, and run it. The next cell shows the code to be run, the header includes the file it will be saved into.

In [ ]:
save_file = tutorial_directory +'0_sample_run.py' 

In [ ]:
%%writetemplate $save_file
{file_header}

print('Hi! This is core number %s out of %s' %(comm.rank, comm.size-1))

# Now we will isolate the task for one processor
if rank == 0:
    print('I am core %s printing this alone!' %(comm.rank))

In the above cell and in the rest of example cells, you will always see **`{file_header}`** at the beginning. This is a variable that is located in the second cell of the document, and contains the lines of code that are always introduced in each code. These lines import basic modules like `sys`, `os`, and also `BigMPI4py`, `MPI4py`, `numpy` and `pandas`. 

The last two lines of this variable are:

`comm = MPI.COMM_WORLD`

`size, rank = comm.Get_size(), comm.Get_rank()`

The first line imports the `comm` object from `MPI4py`, and the second line sets the variables `size`, which contains the number of the processor, and `rank`, which contains the number of cores in which the code is being run. 

The next cell has the console command to run the code. Just launch the cell and the output from console will appear below!

In [ ]:
!mpirun -np $number_cores python $save_file

Now that you know how each block of code is structured, we will add some examples. If you want to change the name of the directory to save the examples of code on, feel save to change the name of `tutorial_directory` and rerun the cell!

## The problem with MPI4py: OverflowError!

The main reason BigMPI4py was developed was because when passing big objects through MPI4py's syntax, it yields an error. Let's imagine we want to divide a big table with 100 rows across 4 cores. The way to do it would be the following:

In [ ]:
save_file = tutorial_directory +'1_sample_run.py'
print(save_file)

A little explanation of the code below if you have never used MPI4py before: we will create a random array `arr` to be scattered through the $n$ different cores, using `MPI4py`. In order to do that we will use `comm.scatter()` function. This function takes a list of $n$ elements, each element being a *chunk* of `arr`; this list will be `list_arr`. 

HOWEVER, `list_arr` will only be generated at the first core (`rank == 0`), so we have to *isolate* the generation of  `arr` and `list_arr` to the first core using `if rank == 0:`. This way, the rest of the cores will skip that part. Still, `list_arr` **must exist** for the rest of cores, since it is being called in `comm.scatter()`. This is the reason why `list_arr = None` is being called at line 8. This is very important: if you don't declare this variable for the rest of cores, the program will yield a `NameError` and the object won't be communicated!

In order to create `list_arr` the common thing to do is to equally divide `arr`. To do that, we create a linspace with the number of rows, that is, a list with $n+1$ positions, where each $x_{i+1} - x_{i}$ in that list has the same value. For instance: linspace(0, 20, 4) -> [0, 5, 10, 15, 20]. If the numbers are not integers, they are converted to integers (some *chunks* will be larger than others, but only by a small difference).

Once `list_arr` is created, it is scattered. For this case, we have created `arr` with $300000000$ cells. This might seem a lot, but nowadays there are many datasets that go beyond this quantity (in scientific research branches like geophysics, astronomy or genomics, or in Big Data common sources like data mining from social networks). 

In [ ]:
%%writetemplate $save_file
{file_header}

list_arr = None 
if rank == 0:
    arr = np.random.rand(30000, 10000)

    lnsp = np.linspace(0, len(arr), {number_cores} + 1)
    list_arr = [arr[int(lnsp[i]):int(lnsp[i+1])] for i in range(len(lnsp) - 1)]

arr_scatter = comm.scatter(list_arr)

print('You have successfully completed this task (although you should not).')

In [ ]:
!mpirun -np $number_cores python $save_file

<span style="color:red; font-size:15px" ><b>STOP THIS CELL BY YOUR OWN USING THE PAUSE</b></span><span style="color:red; font-size:25px"> &#9632; </span><span style="color:red; font-size:15px"><b> BUTTON, OR YOU WON'T BE ABLE TO CONTINUE WITH THE TUTORIAL!</b></span>

When running this last cell, you should encounter an `OverFlowError` (if nothing happens, make the number of rows bigger). This occurs because MPI is run on C, and C works with typed variables, that is, instead of calling `a = 9` like in Python, variables are called using their type: `int a = 9`. By using types, the computer already knows how much memory to allocate and how to work with that variable, making computation much faster than Python, but also *less cleaner*. 

MPI has been developed using an `int` type, whose maximum number can be $2^{31} = 2147483648$. The reason for the error is that, when calling `comm.scatter()`, MPI has to count the elements in `list_arr` (which is much bigger than just only the number of cells in the array!) and, if at some point the counting exceeds that $2^{31}$ value, MPI breaks down, because it can't continue counting more.

What `BigMPI4py` does is subdivide this array (or any other object) into smaller objects and iteratively communicate some of this smaller objects, so that each time the smaller objects are communicated `OverflowError` is avoided. To make things easier, `BigMPI4py` **automatically divides your object** so, insead of passing `list_arr`, you only have to pass `arr`, saving even more of your time!

Having said that... Let's see how to use `BigMPI4py`!

## What can BigMPI4py do?

`BigMPI4py` allows you so far to do most of the common this you could do with `MPI4py` alone. `BigMPI4py` implements 5 functions (which we will add examples of later on): `scatter()`, `bcast()`, `gather()`, `allgather()`, and `sendrecv()`. Before going into examples, we will show what does each function exactly.

The first four functions, `gather()`, `scatter()`, `allgather()`, and `bcast()`, are known as collective communication functions, that is, they involve, in one way or another, all the cores at one moment. `sendrecv()`, on the other hand, is known as a point-to-point communication function, that is, is connects one core to other core at once.

**Scatter**
<img src="Tutorial_figs/scatter.png" alt="scatter" width="60%"/>

`scatter()` communicates an object to the rest of cores, granted the object is divided into $n$ parts to be distributed. At the end of the scattering, each core will have one of the parts. MPI communicates the object so that each part is sequentally scattered, that is, the first core (0) has the first part of teh object, the second core (1) will have the second part, and so on.

**Broadcast**
<img src="Tutorial_figs/broadcast.png" alt="broadcast" width="60%"/>

`bcast()` also communicates an object to the rest of cores, but this time it communicates the whole object to all cores. Thus, at the end of the broadcasting, each core will have an exact copy of the object.

**Gather**
<img src="Tutorial_figs/gather.png" alt="gather" width="60%"/>

`gather()` communicates individual objects, each one in a different core, to a destination core (by default 0). Similar to `scatter()`, the distribution is sequential, so the first element will come from the first core(0), and so on and so forth.

**Allgather**
<img src="Tutorial_figs/allgather.png" alt="allgather" width="60%"/>

`allgather()` is a combination of `gather()` and `bcast()`. `allgather()` takes the object from all the cores, combines it as in `gather()`, and then sends a copy of the object to all cores.

**Sendrecv**
<img src="Tutorial_figs/sendrecv.png" alt="sendrecv" width="60%"/>

`sendrecv()` is a combination of two individual functions: `send()` and `recv()`. `sendrecv()` sends an object from a *source* core to a *destination* core. The rest of the cores will consider this object as `None`.


## Running some examples
We have tried to make the syntax of BigMPI4py as simple as possible. If you have already worked with MPI4py, there will be no problem. If it is your first time running any parallelization on MPI4py or BigMPI4py, looking at the examples will help you understand how to use BigMPI in a matter of minutes.

Instead of loading any data, all the examples will use random objects created in Python. With that, the only required dependencies will be `numpy`, `pandas`, and other modules that are already installed in anaconda. However, you can try to parallelize your own data, feel free to add new cells or modify the existing ones if you prefer! 


### Scattering
We will put some examples of how an array, a dataframe, a simple list (with integers) or a complex list (with arrays) is scattered. For the future cases, we will only choose one of the examples.

<span style="color:red;" ><b>IMPORTANT: So far, lists with mixture of [dataframes, lists, arrays] with [integers, floats, strings] is not implemented!</b></span> 


We will start with one of the common types: a numpy array.

In [ ]:
save_file = tutorial_directory +'2.1_scattering_an_array.py'
print(save_file)

In [ ]:
%%writetemplate $save_file
{file_header}

array = None
if rank == 0:
    array = np.random.rand(2*size , 3)
    print('Rank %s: original array \n%s\n\n' %(rank, array))
    
sc_array = BM.scatter(array, comm)
print('Rank %s: scattered array \n%s\n\n' %(rank, sc_array))


In [ ]:
!mpirun -np $number_cores python $save_file

Now we will scatter a pandas dataframe.

In [ ]:
save_file = tutorial_directory +'2.2_scattering_a_dataframe.py'

In [ ]:
%%writetemplate $save_file
{file_header}

df = None
if rank == 0:
    df = pd.DataFrame((np.random.rand(3*size , 3)*100).astype(str))
    print('Rank %s: original dataframe \n%s\n\n' %(rank, df))
    
sc_df = BM.scatter(df, comm)
print('Rank %s: scattered dataframe \n%s\n\n' %(rank, sc_df))


In [ ]:
!mpirun -np $number_cores python $save_file

Now we will scatter a "simple" list, that is, a list that contains numbers, booleans or strings. Mind that in this example we add mixed types (numpy int and python float) to see that the scattering maintains those types.

In [ ]:
save_file = tutorial_directory +'2.3_scattering_a_simple_list.py'

In [ ]:
%%writetemplate $save_file
{file_header}

lst = None
if rank == 0:
    lst = list(np.arange(3*size)) + [1.2, 1.3, 4.2]
    print('Rank %s: original list \n%s\n'%(rank, lst))
    
sc_lst = BM.scatter(lst, comm)
print('Rank %s: scattered list \n%s\n'%(rank, sc_lst))


In [ ]:
!mpirun -np $number_cores python $save_file

Lastly, we will scater a "complex" list, that is, a list with arrays, dataframes, series or other lists. We will scatter a list with dataframes, and show how they are scattered.

In [ ]:
save_file = tutorial_directory +'2.4_scattering_a_complex_list.py'

In [ ]:
%%writetemplate $save_file
{file_header}

lst = None
if rank == 0:
    lst = []
    for i in range(2*size):
        lst.append(pd.DataFrame(np.ones((2,3))*i))
    print('Rank %s: original list \n%s\n'%(rank, lst))
    
sc_lst = BM.scatter(lst, comm)
print('Rank %s: scattered list \n%s\n'%(rank, sc_lst))


In [ ]:
!mpirun -np $number_cores python $save_file

### Broadcasting
Broadcasting refers to communicating a copy of an object across cores. To make thing easier, we will broadcast an array and a list of arrays.

In [ ]:
save_file = tutorial_directory +'3.1_broadcasting_an_array.py'

In [ ]:
%%writetemplate $save_file
{file_header}

arr = None
if rank == 0:
    for i in range(2*size):
        arr = np.ones((2,3))
        arr = [1,2,2,3]
    print('Rank %s: original array \n%s\n'%(rank, arr))
    
bd_arr = BM.bcast(arr, comm)
print('Rank %s: broadcasted array \n%s\n'%(rank, bd_arr))


In [ ]:
!mpirun -np $number_cores python $save_file

Now we repeat the broadcasting, but with a list of arrays.

In [ ]:
save_file = tutorial_directory +'3.2_broadcasting_a_complex_list.py'

In [ ]:
%%writetemplate $save_file
{file_header}

lst = None
if rank == 0:
    lst = []
    for i in range(size):
        lst.append(np.ones((2,3))*i)
    print('Rank %s: original list \n%s\n'%(rank, lst))
    
bd_lst = BM.bcast(lst, comm)
print('Rank %s: scattered list \n%s\n'%(rank, bd_lst))


In [ ]:
!mpirun -np $number_cores python $save_file

### Gathering and allgathering
The code in gathering is similar to the scattering and broadcasting one; only that it is much simpler! Since all the cores have the object to gather, only one line is needed to do the gathering. As an example, we will gather an array.

In [ ]:
save_file = tutorial_directory +'4.1_gathering_an_array.py'

In [ ]:
%%writetemplate $save_file
{file_header}


arr = np.ones((1,4))*rank # Now all cores have an array.

ga_arr = BM.gather(arr, comm) # After the gathering only the first core has the array.
print('Rank %s: gathered array \n%s\n'%(rank, ga_arr))


In [ ]:
!mpirun -np $number_cores python $save_file

The rest of the cores have `None` as gathered element. If, instead of `gather()` we use `allgather()` all the cores will share the same gatehred array.

In [ ]:
save_file = tutorial_directory +'4.2_allgathering_an_array.py'

In [ ]:
%%writetemplate $save_file
{file_header}


arr = np.ones((1,4))*rank # Now all cores have an array.

ga_arr = BM.allgather(arr, comm) # After the gathering only the first core has the array.
print('Rank %s: allgathered array \n%s\n'%(rank, ga_arr))


In [ ]:
!mpirun -np $number_cores python $save_file

### Point-to-point communication with `sendrecv()`

Using `sendrecv()` is as simple as the other functions. We will do an example with an array, between the first and the second cores.

In [ ]:
save_file = tutorial_directory +'5.1_sendrecv_an_array.py'

In [ ]:
%%writetemplate $save_file
{file_header}

arr = None
if rank == 0: 
    arr = np.ones((3,4))*rank 
    
if rank in [0,1]: print('Rank %s: array to be sent \n%s\n'%(rank, arr))
    
sr_arr = BM.sendrecv(arr, comm, dest=1) # After the gathering only the first core has the array.
if rank in [0,1]: print('Rank %s: recovered array \n%s\n'%(rank, sr_arr))


`dest` argument is the core that will receive the object.

In [ ]:
!mpirun -np $number_cores python $save_file

## More "advanced" stuff
With this examples you should be able to run `BigMPI4py` with any of the supported types. However, if you want to know all the details from MPI4py, we recommend you to continue with this part.

### Scattering an array/dataframe using categorical values

In many cases you have a table that has one or two columns with a categorical value, like a chromosome, a day of the week, an illness, etc. It is common that, when scattering that table, you would want all the rows corresponding to a categorical variable to be at one core, and not distributed across cores. You can do that with `by` argument from `scatter()`. In `by` you have to include a list with the columns (numeric for numpy arrays, numeric or string for pandas dataframes) by which you want to do the distribution. Let's see an example of scattering with and without that argument.

The following example has a set of measurements for some patients, and the day of the week this measurements were done:

In [ ]:
df_meas = '''pd.DataFrame(np.array([['Alfred', 'Mon', 0.1, 23],
                                 ['Alfred', 'Tue', 0.15, 27],
                                 ['Alfred', 'Thr', 0.12, 25],
                                 ['Alfred', 'Thr', 0.13, 18],
                                 ['Alfred', 'Fri', 0.12, 16],
                                 ['Alfred', 'Fri', 0.125, 17],
                                 ['Alfred', 'Fri', 0.118, 15],
                                 ['Bob', 'Mon', 0.24, 20],
                                 ['Bob', 'Mon', 0.17, 23],
                                 ['Bob', 'Tue', 0.14, 16],
                                 ['Bob', 'Tue', 0.16, 18],
                                 ['Charles', 'Mon', 0.13, 20],
                                 ['Charles', 'Mon', 0.16, 23],
                                 ['Charles', 'Tue', 0.14, 22],
                                 ['Charles', 'Fri', 0.15, 27],
                                 ['Charles', 'Fri', 0.16, 24],
                                 ['Charles', 'Fri', 0.15, 22]]),
                                 columns = ['Patient', 'Weekday', 'Meas1', 'Meas2'])
                                 '''


(Don't worry about the dataframe being inside a string. This is done to be communicated to other cells.) You can see that the dataframe has 4 columns: the patient name, the day of the week the measurements were taken, and the two measurements.

The first scattering will be without `by`, to see how is the dataframe scattered.

In [ ]:
save_file = tutorial_directory +'6.1_categorical_scattering_noby.py'

In [ ]:
%%writetemplate $save_file
{file_header}

df = None
if rank == 0: 
    df = {df_meas} 
    
sc_df = BM.scatter(df, comm) 
print('Rank %s: scattered dataframe \n%s\n' %(rank, sc_df))


In [ ]:
!mpirun -np $number_cores python $save_file

You can see that each processor has an almost equal amount of rows per processor. This might not be a problem, but if we wanted to divide the processing by patient name, this would be impossible to do. 

Now, we will perform the scattering using the patient name as the categorical value:

In [ ]:
save_file = tutorial_directory +'6.2_categorical_scattering_by_patient.py'

In [ ]:
%%writetemplate $save_file
{file_header}

df = None
if rank == 0: 
    df = {df_meas} 
    
sc_df = BM.scatter(df, comm, by = ['Patient']) 
print('Rank %s: scattered dataframe \n%s\n' %(rank, sc_df))


In [ ]:
!mpirun -np $number_cores python $save_file

You can see that each processor now only receives one dataframe per patient. If you are running this code with more than 3 cores, then some cores will receive an empty dataframe.

Let's repeat the scattering, although this time using the patient and the weekday as columns.

In [ ]:
save_file = tutorial_directory +'6.3_categorical_scattering_by_patient_and_weekday.py'

In [ ]:
%%writetemplate $save_file
{file_header}

df = None
if rank == 0: 
    df = {df_meas} 
    
sc_df = BM.scatter(df, comm, by = ['Patient', 'Weekday']) 
print('Rank %s: scattered dataframe \n%s\n' %(rank, sc_df))


In [ ]:
!mpirun -np $number_cores python $save_file

This time, each core will receive one or several combinations of patients and weekdays instead.

<span style="color:red;" ><b>WARNING: WHEN USING SCATTERING WITH CATEGORICAL VALUES, MAKE SURE TO SORT THE DATAFRAME/ARRAY FIRST!</b></span> 

If you want to perform a scattering of an array using one or several column, include the index of the columns in the `by` argument.

In [ ]:
save_file = tutorial_directory +'6.4_categorical_scattering_of_array.py'

In [ ]:
%%writetemplate $save_file
{file_header}

arr = None
if rank == 0: 
    arr = np.array([['Alfred', 'Mon', 0.1, 23],
                     ['Alfred', 'Tue', 0.15, 27],
                     ['Alfred', 'Thr', 0.12, 25],
                     ['Alfred', 'Thr', 0.13, 18],
                     ['Bob', 'Mon', 0.24, 20],
                     ['Bob', 'Mon', 0.17, 23],
                     ['Charles', 'Fri', 0.16, 24],
                     ['Charles', 'Fri', 0.15, 22]])

sc_arr = BM.scatter(arr, comm, by = [0, 1]) 
print('Rank %s: scattered array \n%s\n' %(rank, sc_arr))

In [ ]:
!mpirun -np $number_cores python $save_file

### Setting the size limit 
When performing communication with any of the functions, in order to overcome the `OverflowError`, `BigMPI` calculates the size of each `chunk`, that is, how much memory each communicated object consumes. After calculating the size, it also calculates a value, $k$, which is the size of the biggest *chunk* divided by the size limit. Then, each *chunk* will be divided into *subchunks*, and will be communicated. Although theoretically this size limit would be $2^{31}$ **elements** in an array, in reality that size is much smaller, around $10^6$ to $10^7$ **elements** depending on the communicated object (a list, an array, a dataframe, etc.) and the type of the elements within that element (float, int, numpy float, string, mixed types, etc.).

In `BigMPI4py` we have set some values of the size limit by default (between $5\cdot 10^8$ to $1.5 \cdot 10^9$ **bytes**, not elements), depending on the function. We have selected these values trying to minimize the $k$ (the smaller the size limit the bigger the $k$, and the longer the parallelization time might be) but also avoiding it to throw `OverflowError`. We have tried combinations of como data structures and types (numeric or string arrays, dataframes, or list with arrays and dataframes), and none of them have thrown any `OverflowError`.

However, in case this happens to you, don't panick! Just reduce the `size_limit` value and the parallelization will be done. The `size_limit` you have to select for a case is not straightforward to choose, and it is more a trial and error process. A value around $10^6$ to $10^8$ should be fine in almost all cases. Try not to use smaller values (unless it is necessary) because the $k$ value might increase too much and the parallelization time can be long. 

To show how to change the size limit, we will replicate two scenarios: one in which the limit is too big and an `OverflowError` is thrown, and another one in which the size limit is reduced, and no error is thrown. We will scatter a big array in both cases. 

In [ ]:
save_file = tutorial_directory +'7.1_size_limit_throwing_error.py'

In [ ]:
%%writetemplate $save_file
{file_header}

arr = None
if rank == 0: 
    arr = np.random.rand(300000000, 10)

sc_arr = BM.scatter(arr, comm, size_limit = 100000000000) 
print('Rank %s: scattered array \n%s %s\n' %(rank, sc_arr, len(sc_arr)))

In [ ]:
!mpirun -np $number_cores python $save_file

You can see that if `size_limit` is too large, `BigMPI4py` will also throw an `OverflowError`. However, if we reduce this value, the parallelization will be done correctly. 

In [ ]:
save_file = tutorial_directory +'7.2_size_limit_no_error.py'

In [ ]:
%%writetemplate $save_file
{file_header}

arr = None
if rank == 0: 
    arr = np.random.rand(300000000, 10)

sc_arr = BM.scatter(arr, comm, size_limit = 1000000000) 
print('Rank %s: scattered array \n%s %s %s\n' %(rank, sc_arr, len(sc_arr), sc_arr.dtype))

Be patient, it might take some time...

In [ ]:
!mpirun -np $number_cores python $save_file

### `optimize` argument
In `scatter()` and `gather()` you will find the `optimize` argument. If set to `True`, `optimize` will use `MPI4py`'s `comm.Scatterv()` and `comm.Gatherv()` functions to do the scattering and gathering. These functions perform a vectorized communication, that is, they don't directly communicate the object, but the positions where the object will be cut. This method saves a lot of time, but the object to be communicated must be a numeric array.

By default, `scatter()` and `gather()` set `optimize` to `True` and, if the object to be communicated is not a numeric array, the object will be communicated in the traditional way. `optimize` works with any size of arrays. We have seen that, with this method, big arrays (bigger than 300000 to 3000000 of cells, depending on the element type) have around 2 to 5 times smaller communication times than with `optimize` set to `False`. For big arrays this means between seconds to minutes of communication time, making the optimization worth to try.

The next example shows how to activate the optimization. We will compare how long does it take to communicate an arary using the optimized communication VS the not optimized one. 

**Remember, this parameter is only available for `scatter()` and `gather()`.**

In [ ]:
save_file = tutorial_directory +'8.2_optimize.py'
fig_file = tutorial_directory + "timecomp.png"

In [ ]:
%%writetemplate $save_file
{file_header}
import matplotlib.pyplot as plt
import seaborn as sns

arr = None
n_trials = 20
n_rows = 900000

if rank == 0:
    optimized_times, not_optimized_times = [], []


# We first run some tests
for i in range(n_trials):
    if rank == 0: print('Running test %s' %i)
    if rank == 0: arr = np.random.rand(n_rows, 10)
    
    t = time.time()
    sc_arr = BM.scatter(arr, comm, optimize=True) 
    tf_opt = comm.gather(time.time() - t)
    
    if rank == 0: arr = np.random.rand(n_rows, 10)
        
    t = time.time()
    sc_arr = BM.scatter(arr, comm, optimize=False) 
    tf_no_opt = comm.gather(time.time() - t)
    
    
    if rank == 0:
        optimized_times.append(max(tf_opt))
        not_optimized_times.append(max(tf_no_opt))

# And now we plot time differences
if rank == 0:
    fig = plt.figure(figsize = (2.5,4))
    vectorized_list = ["Yes"] * n_trials + ["No"] * n_trials
    time_list = optimized_times + not_optimized_times

    df = pd.DataFrame(dict(Vectorized = vectorized_list, Time = time_list))

    sns.violinplot(x = "Vectorized", y = "Time", data = df, inner="stick")

    plt.savefig("{fig_file}")

Be patient, it might take some time...

In [ ]:
!mpirun -np $number_cores python $save_file

In [ ]:
from IPython.display import display, Image
display(Image(fig_file))

This is the end of the tutorial. 

Look up our paper in bioRxiv to see how the software works.
https://www.biorxiv.org/content/early/2019/01/17/517441

If you find this software useful, please cite us:
Alex M. Ascensión and Marcos J. Araúzo-Bravo. BigMPI4py: Python module for parallelization of Big Data objects; bioRxiv, (2019). doi: 10.1101/517441. 